In [1]:
import pandas as pd
import json

In [6]:
df = pd.read_csv('data/final_clean.csv')

with open('data/test/pred_test.jsonl', 'r') as f:
    predictions = [json.loads(line) for line in f.readlines()]
print(predictions)

[{'ID': 79602891, 'pred_label': 'Natural Sciences'}, {'ID': 82721697, 'pred_label': ''}, {'ID': 66194946, 'pred_label': 'Natural Sciences'}, {'ID': 82966600, 'pred_label': 'Engineering and Technology'}, {'ID': 82966603, 'pred_label': 'Medical and Health Sciences'}, {'ID': 82796480, 'pred_label': ''}, {'ID': 82580093, 'pred_label': 'Natural Sciences'}, {'ID': 82843292, 'pred_label': 'Engineering and Technology'}, {'ID': 80358776, 'pred_label': 'Natural Sciences'}, {'ID': 80457143, 'pred_label': 'Natural Sciences'}, {'ID': 80508628, 'pred_label': 'Humanities'}, {'ID': 82579401, 'pred_label': 'Natural Sciences'}, {'ID': 82503247, 'pred_label': 'Natural Sciences'}, {'ID': 74394934, 'pred_label': 'Natural Sciences'}, {'ID': 82909845, 'pred_label': 'Engineering and Technology'}, {'ID': 82909844, 'pred_label': 'Natural Sciences'}, {'ID': 75131176, 'pred_label': 'Agricultural Sciences'}, {'ID': 80573721, 'pred_label': 'Natural Sciences'}, {'ID': 80534599, 'pred_label': 'Natural Sciences'}, {'I

In [9]:
df_pred = pd.read_json("data/test/pred_test.jsonl", lines=True)
df_pred = df_pred.rename(columns={"ID": "eLIBRARY ID"})


df_main = pd.read_csv('data/final_clean.csv')

df = df_main.merge(df_pred, on="eLIBRARY ID", how="left")

df.dropna(subset=['pred_label'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 624 entries, 1 to 964
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ссылка на статью     624 non-null    object 
 1   DOI                  578 non-null    object 
 2   eLIBRARY ID          624 non-null    int64  
 3   article_type         624 non-null    object 
 4   year                 624 non-null    int64  
 5   language             624 non-null    object 
 6   title_en             624 non-null    object 
 7   key_words_en         624 non-null    object 
 8   abstract_en          624 non-null    object 
 9   authors_metadata_en  624 non-null    object 
 10  title_ru             226 non-null    object 
 11  key_words_ru         226 non-null    object 
 12  abstract_ru          226 non-null    object 
 13  authors_metadata_ru  226 non-null    object 
 14  OECD                 624 non-null    object 
 15  Название журнала     530 non-null    object 


In [14]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


df_clean = df[df['pred_label'].notna() & (df['pred_label'] != "")]

y_true = df_clean['first_OECD']
y_pred = df_clean['pred_label']

# Accuracy
acc = accuracy_score(y_true, y_pred)
print("Accuracy:", acc)

# Classification report (precision, recall, f1-score)
print("\nClassification report:")
print(classification_report(y_true, y_pred))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion matrix:")
print(cm)


Accuracy: 0.1967799642218247

Classification report:
                             precision    recall  f1-score   support

      Agricultural Sciences       0.00      0.00      0.00         6
 Engineering and Technology       0.07      0.24      0.11        29
                 Humanities       0.20      0.17      0.18        42
Medical and Health Sciences       0.17      0.36      0.23        28
           Natural Sciences       0.18      0.52      0.27        95
            Social Sciences       0.76      0.10      0.18       359

                   accuracy                           0.20       559
                  macro avg       0.23      0.23      0.16       559
               weighted avg       0.54      0.20      0.19       559


Confusion matrix:
[[  0   1   0   3   2   0]
 [  1   7   2   2  16   1]
 [  6   3   7   4  17   5]
 [  2   1   0  10  11   4]
 [  8  20   4  12  49   2]
 [ 34  67  22  29 170  37]]


In [13]:
%pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.
